# Matching TCR (servicefönster) to maintenance contracts

We first start by reading the cleaned TCRs for 2023 which includes only maintenance windows (servicefönster).

In [18]:
import pandas as pd

# Define the columns to read
columns_to_read = [
    'TCR-id', 
    'Område',
    'Klassificering', 
    'Från trafikplats',  
    'Från linjespår', 
    'Från inkluderad', 
    'Platssekvensnummer', 
    'Beskrivning',
    'Starttid', 
    'Sluttid', 
    'Servicefönster_nya_kategorier',
    'Relaterade TPÅ:er'
]

# Read the CSV file, selecting only specified columns
csv_file_path = "TCR_T23_X4_servicefönster_cleaned.csv"
tcr_df = pd.read_csv(
    csv_file_path, 
    delimiter=';', 
    usecols=columns_to_read,  # Import only these columns
    on_bad_lines='skip', 
    encoding='utf-8',
    low_memory=False
)


We do some preprocessings:
- set data types
- add column for the duration of the TCRs
- remove cancelled TCRs

In [19]:
# Convert 'Starttid' and 'Sluttid' columns to datetime format
tcr_df['Starttid'] = pd.to_datetime(tcr_df['Starttid'], errors='coerce')
tcr_df['Sluttid'] = pd.to_datetime(tcr_df['Sluttid'], errors='coerce')

In [20]:
# Create a new column 'tid' representing the difference between 'Sluttid' and 'Starttid'
tcr_df['tid_timmar'] = (tcr_df['Sluttid'] - tcr_df['Starttid']).dt.total_seconds() / 3600

we now extract from column Relaterade TPÅ:er, the maintenance contract number and put it in another column.

In [21]:
import re

# List of allowed numbers to keep and their corresponding names
allowed_numbers = {
    "19685": "Malmö och sydöstra Skåne",
    "19686": "Blekinge Kustbana, Kust till Kust banan E2",
    "19352": "Holmsund-Boden S inkl. tvärbanor",
    "19684": "Jönköpingsbanan",
    "19353": "Botniabanan, samt Långsele-Örnsköldsvik-Vännäs samt Forsmo-Hoting",
    "19679": "Länsbanorna i Östergötland och Östra Småland",
    "19351": "Norra Malmbanan",
    "19354": "Södra Malmbanan, Haparandabanan",
    "19680": "Södra Stambanan 1+2, Syd",
    "19468": "Värmland-Dalsland",
    "19469": "Västkustbanan E1, Väst",
    "19683": "Västkustbanan E2, Syd",
    "19470": "Västra Götaland - Göteborg",
    "19485": "Västra Götaland Väst",
    "19486": "Västra Götaland Öst",
    "21283": "Banorna i Bergslagen samt Godsstråket",
    "19325": "Banorna i Bergslagen samt Godsstråket",
    "19326": "Banorna i Bergslagen samt Godsstråket",
    "19654": "Dalabanan, Bergslagspendeln del av Sala-Oxelösund",
    "19653": "Godsstråket",
    "19655": "Stockholm bg, Hagalund",
    "18015": "Kust till kust Väst",
    "19656": "Länsbanorna i Östergötland och Östra Småland",
    "19327": "Mittbanan, Ådalsbanan, samt Forsmo-Hoting",
    "19657": "Mälarbanan",
    "19324": "Norra Stambanan",
    "17948": "Ostkustbanan",
    "22660": "Stockholm bg, Hagalund",
    "22659": "Sthlm Mitt, Citybanan",
    "19658": "Sthlm Mitt, Citybanan",
    "19659": "Stockholm Nord",
    "19660": "Stockholm Syd",
    "19661": "Svealandsbanan, Kolbäck-Rekarne, Eskilstuna-Flen",
    "19662": "Södra Stambanan 1+2, Syd",
    "20354": "Södra Stambanan 1+2, Öst",
    "19664": "Västra Stambanan Hallsberg - Laxå",
    "20353": "Västra Södra Stambanan, Västra stambanan Gnesta - Hallsberg",
    "19663": "Västra Södra Stambanan, Västra stambanan Gnesta - Hallsberg"
}


# Extract the first unique number from 'Relaterade TPÅ:er' and put it in a new column 'Relaterad åtgärdsnummer'
def extract_first_number(value):
    if pd.isna(value):
        return None
    match = re.findall(r'\b(\d+)\b', value)
    if match:
        return match[0]  # Extract the first unique number found
    return None

if 'Relaterade TPÅ:er' in tcr_df.columns:
    tcr_df['Relaterad åtgärdsnummer'] = tcr_df['Relaterade TPÅ:er'].apply(extract_first_number)

# Create a new column 'Relaterad kontrakt' with the corresponding name for 'Relaterad åtgärdsnummer'
tcr_df['Relaterad kontrakt'] = tcr_df['Relaterad åtgärdsnummer'].map(allowed_numbers)

In [22]:
csv_file_path = "TCR_T23_matched.csv"
tcr_df.to_csv(csv_file_path, index=False)

# and for excel
excel_file_path = "TCR_T23_matched.xlsx"
tcr_df.to_excel(excel_file_path, index=False)